### Рамочный ноутбук для ДЗ №5 "Классификация с использованием библиотеки Scikit-learn и интерпретация признаков"

импорт библиотек и определение служебных функций

In [ ]:
%matplotlib inline

import numpy as np
import os, sys
import matplotlib.pyplot as plt
import urllib.request
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
def md5(fname):
    import hashlib
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def show_progress(block_num, block_size, total_size):
    print(round(block_num * block_size / total_size *100,2), end="\r")

In [2]:
def sample_bootstrap(X, y, sample_size, return_OOB_sample = False):
    sample_indices = np.random.randint(0,X.shape[0], sample_size)
    if return_OOB_sample:
        oob_indices = np.array([i for i in np.arange(0, X.shape[0], 1, dtype=int) if i not in sample_indices])
        return X[sample_indices, :], y[sample_indices], X[oob_indices, :], y[oob_indices]
    else:
        return X[sample_indices, :], y[sample_indices]

Следующие две ячейки предназначены для загрузки данных.

In [50]:
# Не меняйте содержимое этой ячейки
# Этот хэш нужен для проверки скачиваемого файла данных

valid_csv_data_hash = '08be44718421aa88ca2742447c480522'

In [5]:
if not os.path.exists('DCIPP_dataset.csv'):
    print('downloading DCIPP data:')
    urllib.request.urlretrieve("https://ml4es.ru/links/dcipp-dataset-csv", "DCIPP_dataset.csv", show_progress)
downloaded_DCIPP_data_hash = md5('./DCIPP_dataset.csv')
assert downloaded_DCIPP_data_hash == valid_csv_data_hash, 'Downloaded DCIPP data is corrupt. Try downloading again.'
print('DCIPP data is valid')

downloading DCIPP data:
DCIPP data is valid


чтение данных и визуальное отображение отдельного экземпляра

In [14]:
df = pd.read_csv('./DCIPP_dataset.csv')

In [51]:
# отобразим первые несколько строк считанных данных
df.head()

,date,year,month,day,nans,range,max,mean,std,svar,...,E5Lq_500[g/kg],E5Lw_850[Pa/s],E5Lw_700[Pa/s],E5Lw_500[Pa/s],E5Ld_950[10^5/s],E5Ld_300[10^5/s],E5Ld_500[10^5/s],E5Lz_500[km],E5Lz_700[km],E5Lz_850[k]
0,1989-05-01,1989,5,1,0,0.4,0.4,0.01481,0.07698,0.00593,...,0.63209,-0.10120,-0.06646,-0.07441,-1.28371,0.84708,-0.21159,5.66631,3.06995,1.49678
1,1989-05-02,1989,5,2,0,4.9,4.9,0.68519,1.35127,1.82593,...,0.97320,-0.11413,-0.15533,-0.19135,-0.79981,1.09787,-0.04874,5.61616,3.03750,1.47336
2,1989-05-03,1989,5,3,0,21.8,21.8,4.30741,4.67456,21.85148,...,0.67237,-0.07146,-0.09326,-0.08508,-0.91296,0.50321,0.19399,5.54298,2.98579,1.43421
3,1989-05-04,1989,5,4,0,5.7,5.7,0.55185,1.55820,2.42798,...,0.57976,-0.01823,0.00713,0.03115,-0.25295,-0.59028,-0.00523,5.54607,2.98530,1.43596
4,1989-05-05,1989,5,5,0,9.0,9.0,1.55556,2.33787,5.46564,...,0.61954,0.00242,0.01053,-0.00686,0.22766,-0.08115,-0.07807,5.49325,2.94660,1.40529


In [16]:
df.columns

Index(['date', 'year', 'month', 'day', 'nans', 'range', 'max', 'mean', 'std',
       'svar', 'kurt', 'skew', 'm2', 'm3', 'm4', 'q_25', 'q_5', 'q_75', 'q_9',
       'q_95', 'q_975', 'q_99', 'q_999', 'urb_mean', 'sub_mean', 'diff',
       'diff_perc', 'E5Stpsmean', 'E5Stpsmax', 'E5Stprange', 'E5Stpstd',
       'E5Stpvar', 'E5Stpkurt', 'E5Stpskew', 'E5Stpm2', 'E5Stpm3', 'E5Stpm4',
       'E5Stpq_25', 'E5Stpq_5', 'E5Stpq_75', 'E5Stpq_9', 'E5Stpq_95',
       'E5Stpq_975', 'E5Stpq_99', 'E5Stpq_999', 'SBCAPE[J/kg]', 'MLCAPE[J/kg]',
       'MUCAPE[J/kg]', 'LLS[m/s]', 'MLS[m/s]', 'DLS[m/s]', 'LR58[K/km]',
       'MLMR05[kg/kg]', 'd_thetaPE[K]', 'PW[kg/m2]', 'TFP_ZTE[-]',
       'DT850[K/km2]', 'GT850[K/km]', 'DPSFC[Pa/km2]', 'GPSFC[Pa/km]',
       'E5SWIND10[m/s]', 'E5SE5SDIVWV[g/m2s]', 'E5Smsl[hPa]',
       'E5LWIND_500[m/s]', 'E5Lt_500[K]', 'E5Lt_850[K]', 'E5Lt_700[K]',
       'E5Lq_500[g/kg]', 'E5Lw_850[Pa/s]', 'E5Lw_700[Pa/s]', 'E5Lw_500[Pa/s]',
       'E5Ld_950[10^5/s]', 'E5Ld_300[10^5/s]'

In [22]:
df.shape

(4339, 77)

In [52]:
# удалим все строки с отсутствующими значениями
df = df.dropna()

In [24]:
df.shape

(3908, 77)

In [54]:
# в признаковом описании оставим только независимые переменные, не связанные напрямую с осадками
X = np.array(df[df.columns[23:]])

# в качестве целевой переменной примем факт превышения 40мм максимальными суточными осадками по 27 станциям региона
y = np.array(df['max']>=40.0)

In [35]:
X.shape, y.shape

((3908, 54), (3908,))

In [55]:
# разведочный анализ данных: выявим соотношение классов

(y==True).sum(), (y==False).sum()

(206, 3702)

In [37]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X, y, test_size=0.25)

In [38]:
Xtrain.shape

(2931, 54)

In [39]:
ytrain.shape

(2931,)

Обучим и оценим качество `примитивной модели` классификации на одном подмножестве обучающих данных

In [46]:
sample_size = Xtrain.shape[0]
Xtr,ytr,Xoob,yoob = sample_bootstrap(Xtrain,ytrain, sample_size=sample_size, return_OOB_sample=True)
model = DummyClassifier(strategy='uniform')
model.fit(Xtr, ytr)
yoob_pred = model.predict(Xoob)
yoob_proba = model.predict_proba(Xoob)
print('accuracy: ' + str(np.mean(np.squeeze(yoob_pred)==yoob)))

accuracy: 0.49813780260707635
